In [15]:
# Load model directly
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("LGAI-EXAONE/EXAONE-Deep-7.8B-AWQ", trust_remote_code=True)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
from transformers import AutoModelForCausalLM
from huggingface_hub import hf_hub_download

# 모델 파일 중 하나의 경로를 가져옵니다 (예: config.json)
model_file = hf_hub_download(repo_id="LGAI-EXAONE/EXAONE-Deep-7.8B-AWQ", filename="config.json")

print("모델 파일 위치:", model_file)

config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

모델 파일 위치: C:\Users\Administrator\.cache\huggingface\hub\models--LGAI-EXAONE--EXAONE-Deep-7.8B-AWQ\snapshots\2c461c2a15579f0e26ffad7e619f3a2ba2ce5e64\config.json


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread

# 모델명 또는 로컬 저장 경로
model_name = "LGAI-EXAONE/EXAONE-Deep-7.8B-AWQ"
streaming = True    # choose the streaming option

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Choose your prompt:
#   Math example (AIME 2024)
prompt = r"""Let $x,y$ and $z$ be positive real numbers that satisfy the following system of equations:
\[\log_2\left({x \over yz}\right) = {1 \over 2}\]\[\log_2\left({y \over xz}\right) = {1 \over 3}\]\[\log_2\left({z \over xy}\right) = {1 \over 4}\]
Then the value of $\left|\log_2(x^4y^3z^2)\right|$ is $\tfrac{m}{n}$ where $m$ and $n$ are relatively prime positive integers. Find $m+n$.

Please reason step by step, and put your final answer within \boxed{}."""
#   Korean MCQA example (CSAT Math 2025)
prompt = r"""Question : $a_1 = 2$인 수열 $\{a_n\}$과 $b_1 = 2$인 등차수열 $\{b_n\}$이 모든 자연수 $n$에 대하여\[\sum_{k=1}^{n} \frac{a_k}{b_{k+1}} = \frac{1}{2} n^2\]을 만족시킬 때, $\sum_{k=1}^{5} a_k$의 값을 구하여라.

Options :
A) 120
B) 125
C) 130
D) 135
E) 140
 
Please reason step by step, and you should write the correct option alphabet (A, B, C, D or E) within \\boxed{}."""

messages = [
    {"role": "user", "content": prompt}
]
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

if streaming:
    streamer = TextIteratorStreamer(tokenizer)
    thread = Thread(target=model.generate, kwargs=dict(
        input_ids=input_ids.to("cuda"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=32768,
        do_sample=True,
        temperature=0.6,
        top_p=0.95,
        streamer=streamer
    ))
    thread.start()

    for text in streamer:
        print(text, end="", flush=True)
else:
    output = model.generate(
        input_ids.to("cuda"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=32768,
        do_sample=True,
        temperature=0.6,
        top_p=0.95,
    )
    print(tokenizer.decode(output[0]))

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread
from awq import AutoAWQForCausalLM  # 여기 핵심
from transformers import AutoTokenizer

# 모델명 또는 로컬 저장 경로
model_name = "LGAI-EXAONE/EXAONE-Deep-7.8B-AWQ"
streaming = True

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)

#  프롬프트 구성
instruction = r"""
Role(역할지정):  
개인정보 식별 및 JSON 변환 전문가로 활동하세요.  

Context(상황):  
- 사용자가 입력한 문장에서 개인정보 항목을 식별하고, 이를 JSON 형식으로 변환해야 합니다.  
- **문장에 포함되지 않은 정보는 null 값이 아닌 빈 배열 []로 출력해야 합니다.**  
- **각 항목별로 여러 개의 데이터가 있을 수 있으며, 이를 리스트(Array) 형태로 저장해야 합니다.**  
- **아래 명시된 JSON 구조 외의 다른 필드(예: "other")는 절대 포함하지 마세요.**  
- **각 항목은 정확한 기준에 따라 매칭해야 하며, 오탐을 방지해야 합니다.**  

1. 이름(name)  
- **여러 개의 이름이 존재하는 경우 리스트(Array) 형태로 반환해야 합니다.**  
  - **"고객명", "성함", "이름"** 등의 키워드가 포함된 값은 반드시 "name" 필드로 저장해야 합니다.  
  - 쉼표(,), ‘및’, ‘와/과’ 같은 연결어로 구분된 경우 각각 개별 요소로 추출  
- **다음 요소는 포함하지 마세요.**  
  - 직급(예: 대표, 부장, 차장, 과장, 사원 등)  
  - 회사명(예: (주)위상, ABC Corp, Google Korea 등)  
  - 존칭(예: 님, 씨, 선생님 등)  
  - 서명 표현(예: 배상, 올림, 드림)  
  - 쉼표(,) 이후 불필요한 정보(예: "김철수, 마케팅팀")  

2. 주소(address)  
- **여러 개의 주소가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  
- **다음 키워드가 포함된 문장을 주소로 인식:**  
  - "건물위치", "주소", "위치", "사업장 주소", "본사 주소", "지점 주소"  
- **다음 요소는 포함하지 마세요.**  
  - 전화번호와 붙어 있는 경우 전화번호 제거  
  - 주소 형식이 아닌 일반적인 지역 언급 제거 (예: "상담 가능 지역: 서울, 경기")  

3. 전화번호(phone_number)  
- **여러 개의 전화번호가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  
- **다음 유형을 포함:**  
  - 국내 휴대전화번호 (예: 010-1234-5678)  
  - 국내 일반전화 (예: 02-1234-5678)  
  - 국제전화 형식 (예: +82-10-1234-5678)  
- **숫자 조합이지만 전화번호가 아닌 경우 제거** (예: "사업자 등록번호 123-45-67890")  

4. 이메일(email)  
- **여러 개의 이메일이 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  
- **이메일 형식을 만족하는 경우만 추출**  

5. 신분증 번호(id_number)  
- **여러 개의 신분증 번호가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  
- **다음 유형을 포함:**  
  - 운전면허증 번호 (예: 12-34-567890-12)  
  - 여권 번호 (예: M12345678)  

6. 주민등록번호(resident_registration_number)  
- **여러 개의 주민등록번호가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  

7. 은행 계좌번호(bank_account)  
- **여러 개의 은행 계좌번호가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  

8. 운송장 번호(tracking_number)  
- **여러 개의 운송장 번호가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  

---

Input Values(입력값):  
- 사용자가 입력한 문장  

Instructions(단계별 지시사항):  
1. 입력된 문장에서 개인정보 항목을 식별하세요.  
2. **각 항목별로 여러 개의 값이 존재할 경우 리스트(Array) 형태로 저장하세요.**  
3. **"other" 필드는 절대 생성하지 마세요.**  
4. **"name" 필드는 "고객명", "성함", "이름" 키워드가 포함된 값을 저장해야 합니다.**  
5. 해당 항목이 존재하면 값을 리스트로 반환하고, 존재하지 않으면 **빈 배열([])**을 할당하세요.  
6. 모든 정보를 JSON 형식으로 변환하세요.  

Constraints(제약사항):  
- **불필요한 필드는 출력하지 않습니다.**  
- **각 항목별 정제 기준을 엄격하게 적용하여 오탐을 방지합니다.**  
- 출력은 반드시 JSON 형식으로 작성하세요.  
- 한국어로 답변하세요.  

---

예제: 필드 정제 후 JSON 출력 (불필요한 "other" 필드 제거)**
"""

input_text = r"""
[심의요청] STUDIO TOMBOY
고객명 : 김민지
휴대폰 : 010-1111-2222
구입일 : 2024-05-17
구매금액 : 85,960원
구매처 :(당사 주문은 주문 번호) SIV / 주문번호 202405173238829
브랜드 : STUDIO TOMBOY
구매점 : SIV
SKU : 9104222391085OS
상품명 : [2403632642] YOUTH 피그먼트 피케 티셔츠
요청내용 :
ㅁ 세탁 후 물 빠짐 / 전체적으로 발생, 흰색 줄 있는 듯한 모양으로 빠짐
"""

# 채팅 메시지 구성
messages = [
    {"role": "system", "content": instruction.strip()},
    {"role": "user", "content": input_text.strip()}
]

# 토큰화
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

# 스트리밍 or 비스트리밍
if streaming:
    streamer = TextIteratorStreamer(tokenizer)
    thread = Thread(target=model.generate, kwargs=dict(
        input_ids=input_ids.to("cuda"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.2,
        top_p=0.95,
        streamer=streamer
    ))
    thread.start()

    for text in streamer:
        print(text, end="", flush=True)
else:
    output = model.generate(
        input_ids=input_ids.to("cuda"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.2,
        top_p=0.95,
    )
    print(tokenizer.decode(output[0], skip_special_tokens=True))

We suggest you to set `torch_dtype=torch.float16` for better efficiency with AWQ.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread
from awq import AutoAWQForCausalLM  # 여기 핵심
from transformers import AutoTokenizer

# 모델명 또는 로컬 저장 경로
model_name = "LGAI-EXAONE/EXAONE-Deep-7.8B-AWQ"
streaming = True

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16  # 또는 float16
).to("cuda")

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)

#  프롬프트 구성
instruction = r"""
Role(역할지정):  
개인정보 식별 및 JSON 변환 전문가로 활동하세요.

Context(상황):  
- 사용자가 입력한 문장에서 개인정보 항목을 식별하고, 이를 JSON 형식으로 변환해야 합니다.  
- 문장에 포함되지 않은 정보는 null 값이 아닌 빈 배열 []로 출력해야 합니다.  
- 각 항목별로 여러 개의 데이터가 있을 수 있으며, 이를 리스트(Array) 형태로 저장해야 합니다.  
- 아래 명시된 JSON 구조 외의 다른 필드(예: "other")는 절대 포함하지 마세요.  
- 각 항목은 정확한 기준에 따라 매칭해야 하며, 오탐을 방지해야 합니다.

1. 이름(name)  
- **여러 개의 이름이 존재하는 경우 리스트(Array) 형태로 반환해야 합니다.**  
  - **"고객명", "성함", "이름"** 등의 키워드가 포함된 값은 반드시 "name" 필드로 저장해야 합니다.  
  - 쉼표(,), ‘및’, ‘와/과’ 같은 연결어로 구분된 경우 각각 개별 요소로 추출  
- **다음 요소는 포함하지 마세요.**  
  - 직급(예: 대표, 부장, 차장, 과장, 사원 등)  
  - 회사명(예: (주)위상, ABC Corp, Google Korea 등)  
  - 존칭(예: 님, 씨, 선생님 등)  
  - 서명 표현(예: 배상, 올림, 드림)  
  - 쉼표(,) 이후 불필요한 정보(예: "김철수, 마케팅팀")  

2. 주소(address)  
- **여러 개의 주소가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  
- **다음 키워드가 포함된 문장을 주소로 인식:**  
  - "건물위치", "주소", "위치", "사업장 주소", "본사 주소", "지점 주소"  
- **다음 요소는 포함하지 마세요.**  
  - 전화번호와 붙어 있는 경우 전화번호 제거  
  - 주소 형식이 아닌 일반적인 지역 언급 제거 (예: "상담 가능 지역: 서울, 경기")  

3. 전화번호(phone_number)  
- **여러 개의 전화번호가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  
- **다음 유형을 포함:**  
  - 국내 휴대전화번호 (예: 010-1234-5678)  
  - 국내 일반전화 (예: 02-1234-5678)  
  - 국제전화 형식 (예: +82-10-1234-5678)  
- **숫자 조합이지만 전화번호가 아닌 경우 제거** (예: "사업자 등록번호 123-45-67890")  

4. 이메일(email)  
- **여러 개의 이메일이 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  
- **이메일 형식을 만족하는 경우만 추출**  

5. 신분증 번호(id_number)  
- **여러 개의 신분증 번호가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  
- **다음 유형을 포함:**  
  - 운전면허증 번호 (예: 12-34-567890-12)  
  - 여권 번호 (예: M12345678)  

6. 주민등록번호(resident_registration_number)  
- **여러 개의 주민등록번호가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  

7. 은행 계좌번호(bank_account)  
- **여러 개의 은행 계좌번호가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  

8. 운송장 번호(tracking_number)  
- **여러 개의 운송장 번호가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  

---

Instructions(지시사항):  
- 입력 문장에서 다음 8가지 항목을 식별하세요:  
  1. name  
  2. address  
  3. phone_number  
  4. email  
  5. id_number  
  6. resident_registration_number  
  7. bank_account  
  8. tracking_number  

- 모든 항목은 반드시 리스트(Array) 형태로 출력해야 하며, 항목이 존재하지 않으면 빈 배열([])로 출력합니다.  
- 절대로 "other" 필드나, 명시되지 않은 기타 필드는 포함하지 마세요.  
- 분석, 해설, 주석 없이 **오직 JSON 코드 블록만 출력하세요.**
- 출력은 반드시 다음과 같은 **순수 JSON 형식의 코드 블록**으로 반환되어야 합니다:  

Constraints(제약사항):  
- **불필요한 필드는 출력하지 않습니다.**  
- **각 항목별 정제 기준을 엄격하게 적용하여 오탐을 방지합니다.**  
- 각 항목은 반드시 리스트([]) 형태로 출력해야 합니다.  
- 해당 항목이 없으면 빈 배열([])로 출력합니다.  
- 불필요한 설명, 분석 과정 없이 **JSON만 출력**해야 합니다.  
- "other" 같은 불필요한 필드는 절대 포함하지 마세요.  
- 출력은 반드시 JSON 형식으로 작성하세요.  
- 답변은 한국어가 아닌 JSON 코드만 반환하세요.

최종 출력은 반드시 아래 형식의 JSON 코드 블록만 포함되어야 합니다.  
그 외의 텍스트, 분석, 해석은 절대 출력하지 마세요.

예시:
```json
{
  "name": [],
  "address": [],
  "phone_number": ["],
  "email": [],
  "id_number": [],
  "resident_registration_number": [],
  "bank_account": [],
  "tracking_number": []
}

"""

input_text = r"""
[심의요청] STUDIO TOMBOY
고객명 : 김민지
휴대폰 : 010-1111-2222
구입일 : 2024-05-17
구매금액 : 85,960원
구매처 :(당사 주문은 주문 번호) SIV / 주문번호 202405173238829
브랜드 : STUDIO TOMBOY
구매점 : SIV
SKU : 9104222391085OS
상품명 : [2403632642] YOUTH 피그먼트 피케 티셔츠
요청내용 :
ㅁ 세탁 후 물 빠짐 / 전체적으로 발생, 흰색 줄 있는 듯한 모양으로 빠짐
"""

# 채팅 메시지 구성
messages = [
    {"role": "system", "content": instruction.strip()},
    {"role": "user", "content": input_text.strip()}
]

# 토큰화
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

# 스트리밍 or 비스트리밍
if streaming:
    # 스트리머에 옵션 추가
    streamer = TextIteratorStreamer(
        tokenizer,
        skip_prompt=True,
        skip_special_tokens=True
    )
    
    # 생성 실행 스레드
    generation_kwargs = dict(
        input_ids=input_ids.to("cuda"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=2048,  # 넉넉하게 확보
        do_sample=False,       # JSON 포맷 정확도 위해 비활성화 (선택)
        temperature=0.2,
        top_p=0.95,
        streamer=streamer
    )
    
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    full_response = ""
    for text in streamer:
        print(text, end="", flush=True)
        full_response += text

    thread.join()  # 스트리밍이 끝날 때까지 기다림
else:
    output = model.generate(
        input_ids=input_ids.to("cuda"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=2024,
        do_sample=False,
        temperature=0.2,
        top_p=0.95,
    )
    
    output_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print(output_text)

C:\ProgramData\Anaconda3\envs\LLM\lib\site-packages\awq\modules\linear\exllama.py:12: UserWarning: AutoAWQ could not load ExLlama kernels extension. Details: DLL load failed while importing exl_ext: 지정된 모듈을 찾을 수 없습니다.
  warnings.warn(f"AutoAWQ could not load ExLlama kernels extension. Details: {ex}")
C:\ProgramData\Anaconda3\envs\LLM\lib\site-packages\awq\modules\linear\exllamav2.py:13: UserWarning: AutoAWQ could not load ExLlamaV2 kernels extension. Details: DLL load failed while importing exlv2_ext: 지정된 모듈을 찾을 수 없습니다.
  warnings.warn(f"AutoAWQ could not load ExLlamaV2 kernels extension. Details: {ex}")
C:\ProgramData\Anaconda3\envs\LLM\lib\site-packages\awq\modules\linear\gemm.py:14: UserWarning: AutoAWQ could not load GEMM kernels extension. Details: DLL load failed while importing awq_ext: 지정된 모듈을 찾을 수 없습니다.
  warnings.warn(f"AutoAWQ could not load GEMM kernels extension. Details: {ex}")
C:\ProgramData\Anaconda3\envs\LLM\lib\site-packages\awq\modules\linear\gemv.py:11: UserWarning:

configuration_exaone.py:   0%|          | 0.00/9.92k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-Deep-7.8B-AWQ:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_exaone.py:   0%|          | 0.00/63.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-Deep-7.8B-AWQ:
- modeling_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
You have loaded an AWQ model on CPU and have a CUDA device available, make sure to set your model on a GPU device in order to run your model.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


...

tokenizer.json:   0%|          | 0.00/4.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/563 [00:00<?, ?B/s]

C:\ProgramData\Anaconda3\envs\LLM\lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\LLM\lib\site-packages\transformers\generation\configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
C:\Users\Administrator\.cache\huggingface\modules\transformers_modules\LGAI-EXAONE\EXAONE-Deep-7.8B-AWQ\2c461c2a15579f0e26ffad7e619f3a2ba2ce5e64\modeling_exaone.py:529: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.f

Okay, let's tackle this problem step by step. The user provided a query where they need to extract specific personal information from a given text and format it into a JSON structure. The instructions are quite detailed, so I need to make sure I follow each guideline precisely.

First, I need to identify all the required fields from the JSON structure provided in the example. The fields are name, address, phone_number, email, id_number, resident_registration_number, bank_account, and tracking_number. Each of these should be an array, even if empty.

Looking at the input text provided:

- **Name**: The text mentions "김민지" which is "Kim Min-ji" in Korean. Since the instruction says to include names with keywords like "고객명", "성함", "이름", this should be under the "name" field as an array. But since it's a single name, it will be a single-element array.

- **Phone Number**: The phone number given is "010-1111-2222". The instructions specify that domestic mobile numbers (starting with 010 or 

KeyboardInterrupt: 

In [3]:
import json
import re

# 예시 출력 문자열 (실제 LLM 모델 출력값을 여기에 붙여넣기)
llm_output = """
{
  "name": ["김민지"],
  "address": [],
  "phone_number": ["010-1111-2222"],
  "email": [],
  "id_number": [],
  "resident_registration_number": [],
  "bank_account": [],
  "tracking_number": []
}
"""

# 결과 검증 함수
def validate_personal_info_json(output: str):
    try:
        # JSON 파싱
        data = json.loads(output)

        # 필수 키 목록
        required_keys = [
            "name", "address", "phone_number", "email", "id_number",
            "resident_registration_number", "bank_account", "tracking_number"
        ]

        # 모든 필드 존재 여부 확인 및 타입 검사
        for key in required_keys:
            if key not in data:
                return f"❌ 누락된 필드: {key}"
            if not isinstance(data[key], list):
                return f"❌ {key} 필드는 리스트(Array)여야 합니다."

        # 불필요한 필드가 포함되어 있는지 확인
        for key in data.keys():
            if key not in required_keys:
                return f"❌ 허용되지 않은 필드가 포함됨: {key}"

        return "✅ JSON 형식 검증 완료: 모든 필드가 정확히 구성되어 있습니다."
    
    except json.JSONDecodeError:
        return "❌ JSON 디코딩 실패: 출력이 올바른 JSON 형식이 아닙니다."

# 검증 실행
validate_personal_info_json(llm_output)

'✅ JSON 형식 검증 완료: 모든 필드가 정확히 구성되어 있습니다.'

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
    "LGAI-EXAONE/EXAONE-Deep-7.8B-AWQ",
    cache_dir="E:/LLM/exaone-7.8b-awq",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(
    "LGAI-EXAONE/EXAONE-Deep-7.8B-AWQ",
    cache_dir="E:/LLM/exaone-7.8b-awq",
    trust_remote_code=True
)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread


# 모델과 토크나이저 로드
model_name = "LGAI-EXAONE/EXAONE-Deep-7.8B-AWQ"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    cache_dir="E:/LLM/exaone-7.8b-awq",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir="E:/LLM/exaone-7.8b-awq",
    trust_remote_code=True
)

#  프롬프트 구성
instruction = r"""
Role(역할지정):  
개인정보 식별 및 JSON 변환 전문가로 활동하세요.

Context(상황):  
- 사용자가 입력한 문장에서 개인정보 항목을 식별하고, 이를 JSON 형식으로 변환해야 합니다.  
- 문장에 포함되지 않은 정보는 null 값이 아닌 빈 배열 []로 출력해야 합니다.  
- 각 항목별로 여러 개의 데이터가 있을 수 있으며, 이를 리스트(Array) 형태로 저장해야 합니다.  
- 아래 명시된 JSON 구조 외의 다른 필드(예: "other")는 절대 포함하지 마세요.  
- 각 항목은 정확한 기준에 따라 매칭해야 하며, 오탐을 방지해야 합니다.

1. 이름(name)  
- **여러 개의 이름이 존재하는 경우 리스트(Array) 형태로 반환해야 합니다.**  
  - **"고객명", "성함", "이름"** 등의 키워드가 포함된 값은 반드시 "name" 필드로 저장해야 합니다.  
  - 쉼표(,), ‘및’, ‘와/과’ 같은 연결어로 구분된 경우 각각 개별 요소로 추출  
- **다음 요소는 포함하지 마세요.**  
  - 직급(예: 대표, 부장, 차장, 과장, 사원 등)  
  - 회사명(예: (주)위상, ABC Corp, Google Korea 등)  
  - 존칭(예: 님, 씨, 선생님 등)  
  - 서명 표현(예: 배상, 올림, 드림)  
  - 쉼표(,) 이후 불필요한 정보(예: "김철수, 마케팅팀")  

2. 주소(address)  
- **여러 개의 주소가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  
- **다음 키워드가 포함된 문장을 주소로 인식:**  
  - "건물위치", "주소", "위치", "사업장 주소", "본사 주소", "지점 주소"  
- **다음 요소는 포함하지 마세요.**  
  - 전화번호와 붙어 있는 경우 전화번호 제거  
  - 주소 형식이 아닌 일반적인 지역 언급 제거 (예: "상담 가능 지역: 서울, 경기")  

3. 전화번호(phone_number)  
- **여러 개의 전화번호가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  
- **다음 유형을 포함:**  
  - 국내 휴대전화번호 (예: 010-1234-5678)  
  - 국내 일반전화 (예: 02-1234-5678)  
  - 국제전화 형식 (예: +82-10-1234-5678)  
- **숫자 조합이지만 전화번호가 아닌 경우 제거** (예: "사업자 등록번호 123-45-67890")  

4. 이메일(email)  
- **여러 개의 이메일이 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  
- **이메일 형식을 만족하는 경우만 추출**  

5. 신분증 번호(id_number)  
- **여러 개의 신분증 번호가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  
- **다음 유형을 포함:**  
  - 운전면허증 번호 (예: 12-34-567890-12)  
  - 여권 번호 (예: M12345678)  

6. 주민등록번호(resident_registration_number)  
- **여러 개의 주민등록번호가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  

7. 은행 계좌번호(bank_account)  
- **여러 개의 은행 계좌번호가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  

8. 운송장 번호(tracking_number)  
- **여러 개의 운송장 번호가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  

---

Instructions(지시사항):  
- 입력 문장에서 다음 8가지 항목을 식별하세요:  
  1. name  
  2. address  
  3. phone_number  
  4. email  
  5. id_number  
  6. resident_registration_number  
  7. bank_account  
  8. tracking_number  

- 모든 항목은 반드시 리스트(Array) 형태로 출력해야 하며, 항목이 존재하지 않으면 빈 배열([])로 출력합니다.  
- 절대로 "other" 필드나, 명시되지 않은 기타 필드는 포함하지 마세요.  
- 분석, 해설, 주석 없이 **오직 JSON 코드 블록만 출력하세요.**
- 출력은 반드시 다음과 같은 **순수 JSON 형식의 코드 블록**으로 반환되어야 합니다:  

Constraints(제약사항):  
- **불필요한 필드는 출력하지 않습니다.**  
- **각 항목별 정제 기준을 엄격하게 적용하여 오탐을 방지합니다.**  
- 각 항목은 반드시 리스트([]) 형태로 출력해야 합니다.  
- 해당 항목이 없으면 빈 배열([])로 출력합니다.  
- 불필요한 설명, 분석 과정 없이 **JSON만 출력**해야 합니다.  
- "other" 같은 불필요한 필드는 절대 포함하지 마세요.  
- 출력은 반드시 JSON 형식으로 작성하세요.  
- 답변은 한국어가 아닌 JSON 코드만 반환하세요.

최종 출력은 반드시 아래 형식의 JSON 코드 블록만 포함되어야 합니다.  
그 외의 텍스트, 분석, 해석은 절대 출력하지 마세요.

예시:
```json
{
  "name": [],
  "address": [],
  "phone_number": ["],
  "email": [],
  "id_number": [],
  "resident_registration_number": [],
  "bank_account": [],
  "tracking_number": []
}

"""

input_text = r"""
[심의요청] STUDIO TOMBOY
고객명 : 김민지
휴대폰 : 010-1111-2222
구입일 : 2024-05-17
구매금액 : 85,960원
구매처 :(당사 주문은 주문 번호) SIV / 주문번호 202405173238829
브랜드 : STUDIO TOMBOY
구매점 : SIV
SKU : 9104222391085OS
상품명 : [2403632642] YOUTH 피그먼트 피케 티셔츠
요청내용 :
ㅁ 세탁 후 물 빠짐 / 전체적으로 발생, 흰색 줄 있는 듯한 모양으로 빠짐
"""

# 채팅 메시지 구성
messages = [
    {"role": "system", "content": instruction.strip()},
    {"role": "user", "content": input_text.strip()}
]

# 토큰화
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

device = "cuda" if torch.cuda.is_available() else "cpu"
input_ids = input_ids.to(device)
model.to(device)

streaming = True

# 스트리밍 or 비스트리밍
if streaming:
    # 스트리머에 옵션 추가
    streamer = TextIteratorStreamer(
        tokenizer,
        skip_prompt=True,
        skip_special_tokens=True
    )
    
    # 생성 실행 스레드
    generation_kwargs = dict(
        input_ids=input_ids.to("cuda"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=2048,  # 넉넉하게 확보
        do_sample=False,       # JSON 포맷 정확도 위해 비활성화 (선택)
        temperature=0.2,
        top_p=0.95,
        streamer=streamer
    )
    
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    full_response = ""
    for text in streamer:
        print(text, end="", flush=True)
        full_response += text

    thread.join()  # 스트리밍이 끝날 때까지 기다림
else:
    output = model.generate(
        input_ids=input_ids.to("cuda"),
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=2024,
        do_sample=False,
        temperature=0.2,
        top_p=0.95,
    )
    
    output_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print(output_text)

In [ ]:
import torch
import time
import json
import psutil
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers import AutoModelForCausalLM, AutoTokenizer


# 모델과 토크나이저 로드
model_name = "LGAI-EXAONE/EXAONE-Deep-7.8B-AWQ"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    cache_dir="E:/LLM/exaone-7.8b-awq",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir="E:/LLM/exaone-7.8b-awq",
    trust_remote_code=True
)

#  프롬프트 구성
instruction = r"""
Role(역할지정):  
개인정보 식별 및 JSON 변환 전문가로 활동하세요.

Context(상황):  
- 사용자가 입력한 문장에서 개인정보 항목을 식별하고, 이를 JSON 형식으로 변환해야 합니다.  
- 문장에 포함되지 않은 정보는 null 값이 아닌 빈 배열 []로 출력해야 합니다.  
- 각 항목별로 여러 개의 데이터가 있을 수 있으며, 이를 리스트(Array) 형태로 저장해야 합니다.  
- 아래 명시된 JSON 구조 외의 다른 필드(예: "other")는 절대 포함하지 마세요.  
- 각 항목은 정확한 기준에 따라 매칭해야 하며, 오탐을 방지해야 합니다.

1. 이름(name)  
- **여러 개의 이름이 존재하는 경우 리스트(Array) 형태로 반환해야 합니다.**  
  - **"고객명", "성함", "이름"** 등의 키워드가 포함된 값은 반드시 "name" 필드로 저장해야 합니다.  
  - **파일명, 제목, 메타데이터에 포함된 형식(예: '브랜드_이름', '작성자: 이름')도 탐지 대상에 포함**  
  - **쉼표(,), ‘및’, ‘와/과’ 같은 연결어로 구분된 경우 각각 개별 요소로 추출** 
- **다음 요소는 포함하지 마세요.**  
  - 직급(예: 대표, 부장, 차장, 과장, 사원 등)  
  - 회사명(예: (주)위상, ABC Corp, Google Korea 등)  
  - 존칭(예: 님, 씨, 선생님 등)  
  - 서명 표현(예: 배상, 올림, 드림)  
  - 쉼표(,) 이후 불필요한 정보(예: "김철수, 마케팅팀")  

2. 주소(address)  
- **여러 개의 주소가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  
- **다음 키워드가 포함된 문장을 주소로 인식:**  
  - "건물위치", "주소", "위치", "사업장 주소", "본사 주소", "지점 주소"  
- **다음 요소는 포함하지 마세요.**  
  - 전화번호와 붙어 있는 경우 전화번호 제거  
  - 주소 형식이 아닌 일반적인 지역 언급 제거 (예: "상담 가능 지역: 서울, 경기")  

3. 전화번호(phone_number)  
- **여러 개의 전화번호가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  
- **다음 유형을 포함:**  
  - 국내 휴대전화번호 (예: 010-1234-5678)  
  - 국내 일반전화 (예: 02-1234-5678)  
  - 국제전화 형식 (예: +82-10-1234-5678)  
- **숫자 조합이지만 전화번호가 아닌 경우 제거** (예: "사업자 등록번호 123-45-67890")  

4. 이메일(email)  
- **여러 개의 이메일이 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  
- **이메일 형식을 만족하는 경우만 추출**  

5. 신분증 번호(id_number)  
- **여러 개의 신분증 번호가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  
- **다음 유형을 포함:**  
  - 운전면허증 번호 (예: 12-34-567890-12)  
  - 여권 번호 (예: M12345678)  

6. 주민등록번호(resident_registration_number)  
- **여러 개의 주민등록번호가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  

7. 은행 계좌번호(bank_account)  
- **여러 개의 은행 계좌번호가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  

8. 운송장 번호(tracking_number)  
- **여러 개의 운송장 번호가 포함될 수 있으므로 리스트(Array)로 반환해야 합니다.**  

---

Instructions(지시사항):  
- 입력 문장에서 다음 8가지 항목을 식별하세요:  
  1. name  
  2. address  
  3. phone_number  
  4. email  
  5. id_number  
  6. resident_registration_number  
  7. bank_account  
  8. tracking_number  

- 모든 항목은 반드시 리스트(Array) 형태로 출력해야 하며, 항목이 존재하지 않으면 빈 배열([])로 출력합니다.  
- 절대로 "other" 필드나, 명시되지 않은 기타 필드는 포함하지 마세요.  
- 분석, 해설, 주석 없이 **오직 JSON 코드 블록만 출력하세요.**
- 출력은 반드시 다음과 같은 **순수 JSON 형식의 코드 블록**으로 반환되어야 합니다:  

Constraints(제약사항):  
- **불필요한 필드는 출력하지 않습니다.**  
- **각 항목별 정제 기준을 엄격하게 적용하여 오탐을 방지합니다.**  
- 각 항목은 반드시 리스트([]) 형태로 출력해야 합니다.  
- 해당 항목이 없으면 빈 배열([])로 출력합니다.  
- 불필요한 설명, 분석 과정 없이 **JSON만 출력**해야 합니다.  
- "other" 같은 불필요한 필드는 절대 포함하지 마세요.  
- 출력은 반드시 JSON 형식으로 작성하세요.  
- 답변은 한국어가 아닌 JSON 코드만 반환하세요.

최종 출력은 반드시 아래 형식의 JSON 코드 블록만 포함되어야 합니다.  
그 외의 텍스트, 분석, 해석은 절대 출력하지 마세요.

예시:
```json
{
  "name": [],
  "address": [],
  "phone_number": ["],
  "email": [],
  "id_number": [],
  "resident_registration_number": [],
  "bank_account": [],
  "tracking_number": []
}

"""

input_text = r"""
[심의요청] STUDIO TOMBOY_최재식
고객명 : 김민지
휴대폰 : 010-1111-2222
구입일 : 2024-05-17
구매금액 : 85,960원
구매처 :(당사 주문은 주문 번호) SIV / 주문번호 202405173238829
브랜드 : STUDIO TOMBOY
구매점 : SIV
SKU : 9104222391085OS
상품명 : [2403632642] YOUTH 피그먼트 피케 티셔츠
요청내용 :
ㅁ 세탁 후 물 빠짐 / 전체적으로 발생, 흰색 줄 있는 듯한 모양으로 빠짐
"""

# 채팅 메시지 구성
messages = [
    {"role": "system", "content": instruction.strip()},
    {"role": "user", "content": input_text.strip()}
]

# 토큰화
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

device = "cuda" if torch.cuda.is_available() else "cpu"
input_ids = input_ids.to(device)
model.to(device)

streaming = True

# 정답 JSON
ground_truth = {
    "name": ["최재식", "김민지"],
    "address": [],
    "phone_number": ["010-1111-2222"],
    "email": [],
    "id_number": [],
    "resident_registration_number": [],
    "bank_account": [],
    "tracking_number": []
}

# 성능 평가 함수
def evaluate_generation(model, tokenizer, input_ids, ground_truth):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    input_ids = input_ids.to(device)
    model.to(device)

    gpu_mem_start = torch.cuda.memory_allocated() if torch.cuda.is_available() else 0
    cpu_mem_start = psutil.virtual_memory().used
    start_time = time.time()

    output = model.generate(
        input_ids=input_ids,
        eos_token_id=tokenizer.eos_token_id,
        max_new_tokens=2048,
        do_sample=False,
        temperature=0.2,
        top_p=0.95,
    )
    output_text = tokenizer.decode(output[0], skip_special_tokens=True)

    end_time = time.time()
    gpu_mem_end = torch.cuda.memory_allocated() if torch.cuda.is_available() else 0
    cpu_mem_end = psutil.virtual_memory().used

    print("\n📦 생성 결과:")
    print(output_text)

    print("\n⏱️ 속도/리소스 측정:")
    print(f"- 처리 시간: {end_time - start_time:.2f}초")
    if torch.cuda.is_available():
        print(f"- GPU 메모리 사용: {(gpu_mem_end - gpu_mem_start) / 1024 ** 2:.2f} MB")
    print(f"- CPU 메모리 사용: {(cpu_mem_end - cpu_mem_start) / 1024 ** 2:.2f} MB")

    try:
        parsed = json.loads(output_text)
        assert isinstance(parsed, dict), "JSON 최상단은 객체여야 함"
        json_valid = True
    except Exception as e:
        json_valid = False
        parsed = {key: [] for key in ground_truth}
        print(f"\n❌ JSON 파싱 실패 또는 형식 오류: {e}")

    print("\n📊 정확도 평가 (Precision / Recall / F1):")
    for key in ground_truth:
        gt = set(ground_truth[key])
        pred = set(parsed.get(key, [])) if json_valid else set()

        tp = len(gt & pred)
        fp = len(pred - gt)
        fn = len(gt - pred)

        precision = tp / (tp + fp) if tp + fp else 1
        recall = tp / (tp + fn) if tp + fn else 1
        f1 = 2 * precision * recall / (precision + recall) if precision + recall else 0

        print(f"- {key}: P={precision:.2f} / R={recall:.2f} / F1={f1:.2f}")

    return parsed

# 평가 함수 실행
evaluate_generation(model, tokenizer, input_ids, ground_truth)